In [2]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import LsiModel
from gensim.matutils import corpus2dense
# from nltk.corpus import stopwords
import numpy as np
from nltk import ngrams
from itertools import groupby
from jieba import cut as jieba_cut
from sklearn.metrics.pairwise import cosine_similarity

#### LSI
```
+--------+     +---+
|        |     |   |
|        |     |   |     +---+     +--------+
|   X    |  =  | U |  *  | S |  *  |    V   |
|        |     |   |     +---+     +--------+
|        |     |   |      TxT       T x |D|
+--------+     +---+
 |V| x |D|     |V|xT
```

#### Method 1
```
+---------+
|         | <-- find the index of max value for topic 1
|    V    |
|         | <-- find the index of max value for topic T
+---------+
  T x |D|
```

#### Method 2
```
+---------+
|         | <-- only keep top 3*|D| values as edge-weights of a graph
|   Sim   |     choose most connected vertexes
|         |
+---------+
 |D| x |D|
```

In [4]:
def preprocess_zh(text):
    text = text.replace('\n', '').replace(' ', '')
    return [[w for w in list(d) if w != u' ']
            for b, d in groupby(list(jieba_cut(text)),
                                lambda s: s == u'。' or s == u'？' or s == u'！') if not b]

In [10]:
def summarize_zh(text, n_summary=None):
    text2 = preprocess(text)
    if n_summary is None:
        n_summary = max(3, len(text2) / 10)
    dic = Dictionary(text2)
    # stop_words = stopwords.words('english')
    # stop_ids = [dic.token2id[w] for w in stop_words if w in dic.token2id]
    once_ids = [i for i, f in dic.dfs.items() if f <= 1]
    # dic.filter_tokens(stop_ids + once_ids)
    dic.filter_tokens(once_ids)
    dic.compactify()

    corpus = [dic.doc2bow(s) for s in text2]
    tfidf = TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    lsi = LsiModel(corpus_tfidf, id2word=dic, num_topics=len(text2) // 2)
    corpus_lsi = lsi[corpus_tfidf]

    U = lsi.projection.u                             # U is |V| * topics
    S = np.diag(lsi.projection.s)                    # S is topics * topics
    V = corpus2dense(corpus_lsi, len(text2) // 2)    # V is topics * |D|
    # X = np.dot(np.dot(U, S), V)

    # auto summarization method 2
    result1 = []
    for dim in range(n_summary):
        i = np.argmax(V[dim])
        result1.append((i, ''.join(text2[i])))

    # auto summarization method 1
    result2 = []
    Sim = cosine_similarity(V)
    threshold = np.sort(Sim.flat)[::-1][n_summary * 8]
    for i in np.argsort([sum(x) for x in (Sim > threshold)])[::-1][:n_summary]:
        result2.append((i, ''.join(text2[i])))

    return (result1, result2, U, S, V)

In [17]:
def rouge_2(human_summary, candidate_summary):
    cd = set(ngrams(candidate_summary, 2))
    score = []
    for s in human_summary:
        hs = set(ngrams(s, 2))
        score.append(len(hs & cd) / float(len(hs)))
    return sum(score)

In [11]:
news1 = u"""剛傳出將低調退出行動市場，並且停止代號為 SoFIA 和 B roxton 的淩動（Atom）處理器開發的英特爾（Intel），在此慘痛的教訓後，也繳出了近百億元美金的學費。只是，一般預料，英特爾推出行動市場，將有利於國內行動晶片大廠聯發科（MediaTek）的發展。但事實是，英特爾傳出推出行動市場的隔天，聯發科的股價卻開盤大跌 8% 。分析師就指出，這其中的關鍵就在與英特爾與中國行動晶片廠展訊的態度。
事實上，英特爾過去在平板電腦市場，透過高額的補貼政策，搶走了聯發科不少訂單。但是，在智慧型手機市場，英特爾雖然有其龐大的晶片設計實力，但是更新速度的落後，加上一直沒有大客戶的支持，因此一直被視為是市場的攪局者。因此，最後英特爾選擇退出，理論上應該可以減輕聯發科的競爭壓力。但是，聯發科的股價在英特爾傳出推出行動市場的消息後隔天，開盤大跌超過 8%，分析師就表示，其中的關鍵因素在展訊（SPREADTRUM ）。
摩根大通（JPMorgan）半導體分析師 Gokul Hariharan 表示，英特爾在 2014年 對展訊投資了 15 億美元的資金，未來英特爾也希望展訊採用其 x86 架構。而展訊執行長李力遊也在 2015 年時表示，2016 年處理器將全部交由英特爾採用其最先進的 14nm 製程來代工，顯示展訊對其基頻技術感到興趣。
Gokul Hariharan 進一步指出，雖然在當前市場中，聯發科儘管已經在 4G 產品的市場上佔據了較大的市佔率，並推出了高端的 Helio X 系列處理器。但是由於其成本較高、基頻技術又有待加強，而且品牌知名度較低的情況下，在高階市場上仍無法挑戰高通的王座，在中低階市場上，2015 年展訊的價格戰，也讓聯發科疲於應付，導致聯發科的毛利率一再調降，使得股價直接被腰斬。因此，如果在 4G 領域被展訊迎頭趕上，將使得聯發科面臨更大考驗。
因此，英特爾即被推出行動通訊市場，但是透過與展訓的合作關係，未來對於聯發科，甚至是高通都還有著長遠的影響。雖然，目前展訊在 4G 市場仍處在試水階段，預計 2016 年下半年才會逐步量產。但是，未來若英特爾將基頻技術授權給展訊，展訊就會成為聯發科直接的競爭對手。
值得一提的是，在英特爾退出行動通訊市場後，華碩（ASUS）的 Zenfone 2 就成了末代採用英特爾行動晶片的智慧型手機。但是，儘管華碩砍掉了部分英特爾移動晶片訂單，英特爾放棄移動晶片的消息仍使得華碩股價在當日下跌了 3.35%  。原因在於市場擔心華碩從英特爾獲得的補貼將因此減少，衝擊其毛利率，造成投資人的疑慮。"""
result1 = summarize_zh(news1)

In [15]:
human_summary1 = [
u"但事實是，英特爾傳出推出行動市場的隔天，聯發科的股價卻開盤大跌8%。",
u"摩根大通（JPMorgan）半導體分析師Gokul Hariharan表示，英特爾在2014年對展訊投資了15億美元的資金，未來英特爾也希望展訊採用其x86架構。",
u"而展訊執行長李力遊也在2015年時表示，2016年處理器將全部交由英特爾採用其最先進的14nm製程來代工，顯示展訊對其基頻技術感到興趣。",
u"因此，英特爾即被推出行動通訊市場，但是透過與展訓的合作關係，未來對於聯發科，甚至是高通都還有著長遠的影響。",
u"但是，未來若英特爾將基頻技術授權給展訊，展訊就會成為聯發科直接的競爭對手。",
]

In [18]:
for i, s in result1[0]:
    print(i, rouge_2(human_summary1, s), s)

16 0.5082536213786214 值得一提的是，在英特爾退出行動通訊市場後，華碩（ASUS）的Zenfone2就成了末代採用英特爾行動晶片的智慧型手機
11 0.6701987595737595 但是由於其成本較高、基頻技術又有待加強，而且品牌知名度較低的情況下，在高階市場上仍無法挑戰高通的王座，在中低階市場上，2015年展訊的價格戰，也讓聯發科疲於應付，導致聯發科的毛利率一再調降，使得股價直接被腰斬
0 0.47381535131535135 剛傳出將低調退出行動市場，並且停止代號為SoFIA和Broxton的淩動（Atom）處理器開發的英特爾（Intel），在此慘痛的教訓後，也繳出了近百億元美金的學費


In [19]:
for i, s in result1[1]:
    print(i, rouge_2(human_summary1, s), s)

8 1.4774200799200798 摩根大通（JPMorgan）半導體分析師GokulHariharan表示，英特爾在2014年對展訊投資了15億美元的資金，未來英特爾也希望展訊採用其x86架構
4 0.5515030802530803 事實上，英特爾過去在平板電腦市場，透過高額的補貼政策，搶走了聯發科不少訂單
5 0.52120004995005 但是，在智慧型手機市場，英特爾雖然有其龐大的晶片設計實力，但是更新速度的落後，加上一直沒有大客戶的支持，因此一直被視為是市場的攪局者


In [21]:
news2 = u"""英國今天周四進行倫敦市長，市議會投票選舉。這不僅是傳統對立的英國工黨和保守黨的對決，也是有不同身份背景的候選人對決。工黨候選人穆斯林公交車司機的兒子薩迪克汗選前民調領先，有可能擊敗保守黨對手，猶太金融大亨的兒子 紮克 戈德史密斯。兩位熱門後續人今天早上當地時間6點已經投票。另外，今天蘇格蘭，威爾士和北愛爾蘭也舉行國會地方選舉。
今天倫敦市長選舉格外引人著名，這主要是倫敦市長兩個熱門候選人身份截然不同，他們也代表着不同階層的選民。他們兩人在建設低廉價格住房，改善倫敦公交和減少污染等議題上做出不同承諾。
根據5月4日TNSGlobal最新民調顯示，工黨國會議員薩迪克汗擁有57% 的支持率，比對手保守黨候選人戈德密斯高出超過10個百分點，很可能成為倫敦選出的首位穆斯林市長。本次倫敦市長競選活動中，關注候選人身份實屬罕見。分析人士指出，如果戈德史密斯在此次 競選倫敦市長失敗，也代表着他在競選活動中咄咄逼人競選策略的失敗。而薩迪克汗當選也代表着倫敦這個國際都市的真實面貌，就是倫敦1/8的人口是穆斯 林，30%是有色人種，倫敦有寬容的美稱，不同種族信仰的居民希望和平相處。
薩迪克汗今年45歲，他父母來自巴基斯坦，他本人從小在自倫敦南部郊區政府廉價租房社區長大，在8個孩子中他排行5。他為人低調，在競選中表示無論有何種信仰，無論是億萬富翁還是公交司機，他最希望為倫敦人服務，成為所有倫敦人的市長。
而與此相反，對手保守黨候選人戈德史密斯給人一種“富有而且非常帥”的感覺。他今年41歲，曾經就讀於著名伊頓學校，父親是金融界相當成功。戈德史密斯還受 到英國首相卡梅倫的大量支持。而且在競選中戈德史密斯採取“特朗普”方式的競選策略，他咄咄逼人，他用伊斯蘭教信仰作為攻擊競選對手薩迪克汗的主要武器。
法國媒體指出，英國競選活動關注候選人身份政治。在競選活動期間，競選雙方經歷了關於宗教，種族主義議題的辯論，較量。戈德史密斯不斷攻擊薩迪克汗的信仰問 題，並指責他為極端分子辯護，給他們提供表達空間。支持戈德史密斯的英國媒體也呼籲，大家今天會投票給這位工黨候選人，讓他領導倫敦，而同現在，我們共同 的敵人就是大伊斯蘭極端勢力。
薩迪克汗對此進行回擊，表示他自己為有穆斯林信仰驕傲，同時自己從事律師工作，是與極端主義作鬥爭。他指出接受精英教育的戈德史密斯，是代表億萬富翁金融階層利益，企圖用信仰不同來畫分倫敦。
工黨的穆斯林國會議員薩迪克汗今天選舉有望勝出，成為倫敦首位有穆斯林背景的市長， 領導西方大都市倫敦。
當法新社記者問薩迪克汗，如果他勝選後，擔任歐洲第一位有穆斯林背景的市長如何看待？他回答，宗教信仰只是的一部分，我是倫敦人，我也是英國人，我祖籍是巴 基斯坦人。我是一名父親，我是一名丈夫，我從來都是利物浦足球迷。我從競選第一天開始就採取積極的態度。我向選民講述了自己的經歷和經驗。
英國選民今天參加市長和市議會的投票活動。今天晚上當地時間9點投票站關門。同時蘇格蘭，威爾士和北愛爾蘭今天也舉行國會地方選舉，將更換124個市政府的議員席位。"""
result2 = summarize_zh(news2, 5)

In [22]:
human_summary2 = [
u"英國今天周四進行倫敦市長，市議會投票選舉。",
u"今天倫敦市長選舉格外引人著名，這主要是倫敦市長兩個熱門候選人身份截然不同，他們也代表着不同階層的選民。",
u"薩迪克汗今年45歲，他父母來自巴基斯坦，他本人從小在自倫敦南部郊區政府廉價租房社區長大，在8個孩子中他排行5。",
u"而與此相反，對手保守黨候選人戈德史密斯給人一種“富有而且非常帥”的感覺。",
u"在競選活動期間，競選雙方經歷了關於宗教，種族主義議題的辯論，較量。"
]

In [23]:
for i, s in result2[0]:
    print(rouge_2(human_summary2, s), s)

1.3231395757646782 今天倫敦市長選舉格外引人著名，這主要是倫敦市長兩個熱門候選人身份截然不同，他們也代表着不同階層的選民
0.0 我是一名父親，我是一名丈夫，我從來都是利物浦足球迷
0.5426206794872279 分析人士指出，如果戈德史密斯在此次競選倫敦市長失敗，也代表着他在競選活動中咄咄逼人競選策略的失敗
0.5481425055666237 根據5月4日TNSGlobal最新民調顯示，工黨國會議員薩迪克汗擁有57%的支持率，比對手保守黨候選人戈德密斯高出超過10個百分點，很可能成為倫敦選出的首位穆斯林市長
0.14347826086956522 兩位熱門後續人今天早上當地時間6點已經投票


In [24]:
for i, s in result2[1]:
    print(i, rouge_2(human_summary2, s), s)

5 1.3231395757646782 今天倫敦市長選舉格外引人著名，這主要是倫敦市長兩個熱門候選人身份截然不同，他們也代表着不同階層的選民
0 1.077563576702215 英國今天周四進行倫敦市長，市議會投票選舉
12 0.2863372939215115 他為人低調，在競選中表示無論有何種信仰，無論是億萬富翁還是公交司機，他最希望為倫敦人服務，成為所有倫敦人的市長
11 1.0746103363412634 薩迪克汗今年45歲，他父母來自巴基斯坦，他本人從小在自倫敦南部郊區政府廉價租房社區長大，在8個孩子中他排行5
10 0.28816454523803225 而薩迪克汗當選也代表着倫敦這個國際都市的真實面貌，就是倫敦1/8的人口是穆斯林，30%是有色人種，倫敦有寬容的美稱，不同種族信仰的居民希望和平相處
